In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import re

/home/holy/anaconda3/envs/nlp_text/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


## Pre-processing

### Functions

In [7]:
def remove_stop_words(contents):
    stop_words = list(stopwords.words('english'))
    for w in stop_words:
        contents = contents.replace(w, '')
        
    return contents

#### Tokenize and remove unnecessary characters

In [8]:
def remove_unnecessary_characters(contents):
    contents = contents.replace('\n', ' ')
    contents = contents.replace('..', '')
    contents = contents.replace('--', '')
    contents = contents.replace('==', '')
    contents = contents.replace('///', '')
    contents = contents.replace('\\\\', '')
    contents = ' '.join(contents.split())
    contents = contents.strip().lower()
    
#     contents = remove_stop_words(contents)
    tokenizer = RegexpTokenizer('[A-Za-z0-9\@\.\&\/\:\$\-\_]+')
    tokens = tokenizer.tokenize(contents)
    
    tokens = ' '.join( [i for i in tokens if len(i) > 1])
    
    return tokens

In [9]:
def replace_email(content):
    pattern = re.compile('[\w\/\.\-]+\@[\w\/\.\-]+\.[\w]+')
    replaced_content = re.sub(pattern, 'this_is_email', content)
    return replaced_content

In [10]:
def replace_link(content):
    pattern = re.compile('(http[s]?:\/\/|www\.)?[\w\/\.\-]+\.(com|html|php)([\/][\w\/\.\-]*)*')
    replaced_content = re.sub(pattern, 'this_is_link', content)
    return replaced_content

### Main program

#### Read CSV data for train data and test data

In [11]:
train_data = pd.read_csv('dataset/train_data.csv')
test_data = pd.read_csv('dataset/test_data.csv')

#### Tokenize remove unnecessary characters for train data and test data

In [12]:
preproc_train_data = train_data['content'].copy().apply(remove_unnecessary_characters).apply(replace_email).apply(replace_link)

print('Train data')
preproc_train_data.head()

Train data


0    daytips poem-a-day: 09/13/02 sponsor child tod...
1    jody sent you messagejody sent you message. tr...
2    re: tricky perl question ascending orderjozsi ...
3    this_is_email to unsubscribe email to this_is_...
4    re: re moment of silence for the first amendme...
Name: content, dtype: object

In [13]:
preproc_test_data = test_data['content'].copy().apply(remove_unnecessary_characters).apply(replace_email).apply(replace_link)

print('Test data')
preproc_test_data.head()

Test data


0    re: acroread not seeing printerson thu 2010-04...
1    america great misleaderurl: this_is_link 86740...
2    lowestprices guaranteed on flea and tick meds ...
3    re: problems with apt-get -f install once upon...
4    ack apt-get still failing for me stumped. rh8 ...
Name: content, dtype: object

In [17]:
corpus = []

In [18]:
for row in preproc_train_data.iteritems():
    corpus.append(row[1])

In [19]:
for row in preproc_test_data.iteritems():
    corpus.append(row[1])

In [20]:
import gensim

In [21]:
model_cbow = gensim.models.Word2Vec(
    corpus,
    size=50,
    sg=0,
    window=2,
    min_count=5,
    workers=4
)

In [22]:
model_cbow.train(corpus, total_examples=len(corpus), epochs=50)

(79087484, 384610850)

In [23]:
model_sg = gensim.models.Word2Vec(
    corpus,
    size=50,
    sg=1,
    window=2,
    min_count=5,
    workers=4
)

In [25]:
model_sg.train(corpus, total_examples=len(corpus), epochs=50)

(79095938, 384610850)